In [1]:
import pandas as pd # 테이블 형식으로 데이터를 처리할 수 있게 하는 라이브러리

In [2]:
!pip install selenium --quiet # 필수 라이브러리 설치 (한번 설치되면 실행시키지 않아도 됨)

In [3]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
import time

In [27]:
class Covid:
    def __init__(self, url, loop):
        self.url = url
        self.loop = loop,
        self.df = pd.DataFrame()
        self.driver = None
        self.table  = {}
        
    def setup(self, path):
        # 웹드라이버 생성
        self.driver = wd.Chrome(executable_path=path) # 웹드라이버 경로를 executable_path에 설정
        self.driver.get(self.url)
    
    def scroll_covid_table(self, old=False):
        # old 옵션은 확진자 현황 1 ~ x만번까지 집계되어있는 다른 버튼을 누르는 옵션
        if old:
            old_corona_xpath = '//*[@id="move-cont1"]/div[2]/div[4]/div[1]/ul/li[2]/button'
            self.driver.find_element_by_xpath(old_corona_xpath).click()
        else:
            current_corona_xpath = '//*[@id="container"]/div[2]/div/div[4]/ul/li[1]/button'
            self.driver.find_element_by_xpath(current_corona_xpath).click()
    
    def select_location(self):       
        location_button_xpath = '//*[@id="status_searchArea"]/option[3]' #강동구
        self.driver.find_element_by_xpath(location_button_xpath).click()
    
    def start_search(self):
        search_button_xpath = '//*[@id="status_searchButton"]'
        self.driver.find_element_by_xpath(search_button_xpath).click()
        
        time.sleep(3) # 로딩 기다리기
        
    def get_table(self):
        
        target_table_xpath = '//*[@id="DataTables_Table_0"]'
        table = self.driver.find_element_by_xpath(target_table_xpath)

        sets = [table.text][0].split('\n')

        self.table['table_name'] = sets[0].split(' ')[1:]
        self.table['idx'] = [sets[i] for i in range(1, len(sets),2)]
        self.table['data'] = [sets[i] for i in range(2, len(sets), 2)]
            
    def make_table(self):
        # 테이블로 만둘어주는 함수
        df_tmp = pd.DataFrame()
        
        for x in range(1, self.loop[0] + 1):
            
            Covid.get_table(self)
            
            data_tmp = [i.split(' ') for i in self.table['data']]
            for i in data_tmp:
                i.insert(4, ''.join(i[4:-1]))
                del i[5:-1]

            self.table.update({'data' : data_tmp})

            columns = self.table['table_name']
            index = self.table['idx']
            self.df = pd.DataFrame(data_tmp, columns=columns, index=index).reset_index().rename(columns={'index':'연번'})
        
            df_tmp = pd.concat([df_tmp, self.df], axis=0)
            
        self.driver.quit() 
        
        return df_tmp.reset_index(drop=True)

**chromedriver_path는 설치한 웹드라이버 경로로 변경**  
**loop로 얼마만큼의 페이지를 가져올것인지 설정 (최대 27페이지, 한 페이지당 100개 row)**

In [30]:
url = "https://www.seoul.go.kr/coronaV/coronaStatus.do"
chromedriver_path = "C:/chromedriver.exe" #크롬 웹드라이버 경로
loop = 5 #반복횟수(얼마만큼의 페이지 데이터를 가져올건지 선택)

**실행 코드**

In [30]:
gangdong = Covid(url = url, loop = loop)
gangdong.setup(path = chromedriver_path)
gangdong.scroll_covid_table()
gangdong.select_location()
gangdong.start_search()
table = gangdong.make_table()

**테이블 확인**

In [ ]:
table #테이블 확인
#table.head(5) # 위에서 5개 데이터 확인

**저장**  
**path는 저장하고싶은 경로로 변경**

In [32]:
path = './covid.xlsx' #(./ 는 소스코드가 있는 현재 위치를 말함)
table.to_excel(path) #저장